In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import sys
sys.path.append('../modules/')
from mymodule import *
from mymodule2 import *
from pulp_func import *

df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [2]:
merged_df = get_merged_df(df21, df22)
# print(merged_df)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)
expected_sales_2023 = total_yield_2023.copy()
expected_sales_2023.columns = ['作物编号', '作物名称', '预期销售量/斤']
expected_sales = total_yield_2023.copy()
expected_sales.columns = ['作物编号', '作物名称', '预期销售量/斤']
std_devs = [0.005, 0.005, 0.01, 0.01, 0.003, 0.003, 0.003, 0.003]

In [7]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

In [8]:
variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)

In [9]:
# 定义目标函数的各个部分
k = 0.5
min_area_percent = 0.2
max_plots = 4
profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields, k)
# 将目标函数添加到问题中
prob += profit
# print(profit)
prob = add_constraints2(prob, variables, binary_variables, actual_sales, excess_yield,
                        crops, new_fields, total_yield_2023, df2023_result, min_area_percent, max_plots)

In [10]:
# 求解问题
prob.solve()
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [11]:
update_new_fields(variables, new_fields)
# 打印结果
# for field in new_fields:
#     print(field.field_name, field.season, field.planted_crop)
df_template = pd.read_excel('../excels/附件3/template.xlsx')
output(df_template, 2024, new_fields, k)
# print(df_template)

e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
